# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

#### Created by Silvia Huang

## Set up and Import Libraries

In [1]:
#install libraries if needed
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest
from datetime import datetime

In [3]:
#set the root path, change the directory into the project folder
os.chdir("/Users/silvia/cffs-label")

In [4]:
#enable reading data in the scrolling window 
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Import Preprocessed Datasets

In [5]:
Items = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv"))
Items.dtypes

ItemId             object
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [6]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-4271,APPLES GRANNY SMITH,113.0,ea,1.0,CT,PRODUCE
1,I-4971,ARTICHOKE 1/4 SALAD CUT TFC,6.0,LG CAN,2.5,Kg,PRODUCE
2,I-2305,BACON PANCETTA,1.0,Kg,1.0,Kg,MEAT
3,I-1207,BAGUETTE FRENCH,24.0,each,1.0,CT,BREAD
4,I-17203,BALSAMIC GLAZE,2.0,bottle,2.0,L,FOOD - GROCERY


In [7]:
Ingredients = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv"))
Ingredients.dtypes

IngredientId     object
Qty             float64
Uom              object
Conversion      float64
InvFactor       float64
Recipe           object
dtype: object

In [8]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-18746,1.00,Kg,1.00000,1.0000,P-10241
1,I-3388,1.00,L,1.00000,0.3058,P-10496
2,I-4660,2.27,Kg,2.20462,0.6942,P-10496
3,I-3451,2.56,L,1.00000,1.2800,P-13933
4,I-4679,1.00,BUNCH,1.00000,0.0063,P-18318


In [9]:
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv"))
Preps.dtypes

PrepId             object
Description        object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [10]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,NaN
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,NaN
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,NaN


In [11]:
Products = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv"))
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [12]:
Products.head()

,ProdId,Description,SalesGroup
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO
3,R-50859,ALF|Flatbread|Bruschetta,OK - AL FORNO
4,R-50788,ALF|Flatbread|Caprese,OK - AL FORNO


In [13]:
Conversions = pd.read_csv(os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv"))
Conversions.dtypes

ConversionId       object
Multiplier        float64
ConvertFromQty    float64
ConvertFromUom     object
ConvertToQty      float64
ConvertToUom       object
dtype: object

In [14]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,NaN,1.000000,1.0,XXX,1.00,L
1,NaN,0.877193,1.0,1.14L,1.14,L
2,NaN,0.666667,1.0,1.5L,1.50,L
3,NaN,0.571429,1.0,1.75 L,1.75,L
4,NaN,0.500000,1.0,2L,2.00,L


In [15]:
ghge_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "ghge_factors.csv"))
ghge_factors.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [16]:
ghge_factors.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


### Unit Converter

In [17]:
#import standard unit conversion information
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "standard_conversions.csv"))
Std_Unit.head()

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,4.92890,1,tsp,4.92890,ml
1,14.78700,1,Tbsp,14.78700,ml
2,946.35000,1,qt,946.35000,ml
3,473.17625,1,pt,473.17625,ml
4,28.34950,1,oz,28.34950,g


In [18]:
liquid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'ml', 'ConvertFromUom'].tolist()
solid_unit = Std_Unit.loc[Std_Unit['ConvertToUom'] == 'g', 'ConvertFromUom'].tolist()

In [19]:
#construct a standard unit converter
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        Qty = float(qty)*float(multiplier)
        Uom = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'ConvertToUom'].values[0]
    else:
        Qty = qty
        Uom = uom
    return (Qty, Uom)

In [20]:
#test the std_converter
assert std_converter(0.25,'lb') == (113.398, 'g')

In [21]:
#construct a unit converter for specific ingredients
spc_cov = list(filter(None, Conversions['ConversionId'].tolist()))

def spc_converter(ingre, qty, uom):
    if uom in liquid_unit + solid_unit:
        return std_converter(qty, uom)
    elif ingre in spc_cov:
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)
                                    & (Conversions['ConvertToUom'] == 'g')]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else: 
            Qty = float(qty)/float(multiplier)
            Uom = conversion['ConvertToUom'].values[0]
            return (Qty, Uom)
    else:
        return std_converter(qty, uom)

In [22]:
#test the spc_converter
assert spc_converter('I-1120', 1, 'CT') == (50, 'g')

## Clean Preps Units

In [23]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

In [24]:
for index in Preps.index:
    PrepId = Preps.loc[index,'PrepId']
    Qty = Preps.loc[index,'PakQty']
    Uom = Preps.loc[index,'PakUOM']
    Preps.loc[index,'StdQty'] = spc_converter(PrepId, Qty, Uom)[0]
    Preps.loc[index,'StdUom'] = spc_converter(PrepId, Qty, Uom)[1]

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-55516,BAKED|Lasagna|Spin Mushroom,5.550,Kg,NaN,5550.000000,g
1,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,NaN,6176.000000,g
2,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,NaN,7360.000000,g
3,P-56502,BAKED|Pasta|Shrimp Pesto,5.760,Kg,NaN,5760.000000,g
4,P-56433,BATCH|Shrimp Remoulade,1.600,Kg,NaN,1600.000000,g
5,P-50500,CHIFFONADE|Basil,190.000,g,NaN,190.000000,g
6,P-24750,CHOPPED|Cilantro,0.500,Kg,NaN,500.000000,g
7,P-53735,CHOPPED|pickle,900.000,g,NaN,900.000000,g
8,P-58450,COOKED|Bahn Mi Chicken,8.500,Kg,NaN,8500.000000,g
9,P-26068,COOKED|Caramelized Onion,1.200,Kg,PREP,1200.000000,g


### Get Preps with Nonstandard Unit

In [26]:
col_names = list(Preps.columns.values)
Preps_Nonstd = []

for index, row in Preps.iterrows():
    StdUom = Preps.loc[index,'StdUom']
    if StdUom not in ['g', 'ml']:
        Dict = {}
        Dict.update(dict(row))
        Preps_Nonstd.append(Dict)

Preps_Nonstd = pd.DataFrame(Preps_Nonstd, columns = col_names)

In [27]:
Preps_Nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,8.0,each
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,185.0,ea
2,P-51992,YIELD|Bread|Sourdough 5/8,36.0,slice,NaN,36.0,slice
3,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,16.0,ea
4,P-26170,GRILLED|NaanBread,1.0,ea,PREP,1.0,ea
5,P-16305,YIELD|Smokie (1pc),1.0,ea,NaN,1.0,ea
6,P-26047,BOILED|Hard Boiled Eggs FT,50.0,ea,PREP,50.0,ea
7,P-26631,BRK|Pancake|Chocolate Chip,24.0,ea,NaN,24.0,ea
8,P-26057,FRIED|Sunny/Overeasy Egg,1.0,ea,ZDONT USE OK - PREP,1.0,ea
9,P-26056,GRILLED|Tomato,1.0,ea,ZDONT USE OK - PREP,1.0,ea


In [28]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_NonstdUom.csv")
Preps_Nonstd.to_csv(path, index = False, header = True)

### Update Correct Uom for Preps

In [29]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

***

## Add New Items to Database

In [30]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [31]:
Items_Assigned.shape

(778, 8)

### Get the List of New Items

In [32]:
col_names = list(Items.columns.values)
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [33]:
New_Items.insert(1, "CategoryID", '')
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [34]:
New_Items.shape

(0, 8)

In [35]:
# store the list of new items into .csv file
if not New_Items.empty:
    path = os.path.join(os.getcwd(), "data", "mapping", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
    New_Items.to_csv(path, index = False, header = True)

### Import List of New Items with Emission Factors Category ID Assigned

In [36]:
#select the file path for new items list with category id
New_Items_Added = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "new items added", "New_Items_Added_4.csv"))
New_Items_Added

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3838,1,"BEEF BRISKET BNLS 3/4"" AAA",1.000,Kg,1.00,Kg,MEAT
1,I-49128,1,HOT DOG BEEF G/FREE 6 INC,10.000,lb,6.00,CT,MEAT
2,I-17055,1,"BEEF RIB BRAISING 3 BONE 1/4""",1.000,Kg,1000.00,g,MEAT
3,I-2319,3,SMOKIES 5.5 CT BAVARIAN,1.000,Kg,1.00,Kg,MEAT
4,I-42350,3,PORK BACK RIB 1/2 RACK FZ,1.000,Kg,1.00,Kg,MEAT
5,I-3853,3,PORK CHOP 170G C/C,28.000,CT,1.00,CT,MEAT
6,I-8140,3,SAUSAGE CHORIZO WINE,1.000,Kg,10.00,ea,MEAT
7,I-57497,3,SAUSAGE FRANKFURTER FLD ROAST,12.000,pak,4.00,each,MEAT
8,I-22916,3,PORK BELLY SKON COV,1.000,Kg,1.00,Kg,MEAT
9,I-57496,3,SAUSAGE SMKD APPLE FLD ROAST,12.000,pak,4.00,each,MEAT


In [37]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [38]:
Items_Assigned_Updated.shape

(778, 8)

In [39]:
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [40]:
path = os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

## Mapping Items to GHG Factors

In [41]:
df1 = pd.DataFrame(Items_Assigned_Updated)
df2 = pd.DataFrame(ghge_factors)

In [42]:
df1.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-10869,1,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
3,I-37005,1,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT


In [43]:
df1.dtypes

ItemId             object
CategoryID          int64
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [44]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [45]:
df2.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [46]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [47]:
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0

### Manully Adjust GHG Factor for Specific Items

In [48]:
Manual_GHG = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Manual_Adjust_GHGe.csv"))
Manual_GHG.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-52090,59,BURGER BEEF & MUSHROOM HALAL,1.0,cs,48.00,CT,MEAT,59,manually adjusted,31.13750
1,I-45558,59,Prep-Vegan Parmesan,1000.0,g,1.00,g,PRODUCTION FOOD,59,manually adjusted,3.85686
2,I-3352,59,MAYONNAISE PAIL TFC 4L,2.0,each,4.00,L,FOOD - GROCERY,59,manually adjusted,3.55000
3,I-3223,59,COCONUT MILK 17/19% MILK FAT,6.0,LG CAN,2.84,L,FOOD - GROCERY,59,manually adjusted,3.50000
4,I-2898,59,MUSTARD DIJON WINE FLEUR,6.0,jar,1.00,Kg,FOOD - GROCERY,59,manually adjusted,3.32600


In [49]:
for index, row in Manual_GHG.iterrows():
    itemId = Manual_GHG.loc[index, 'ItemId']
    ghge = Manual_GHG.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge

In [50]:
mapping#.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1,CHUCK FLAT BONELESS FZN,3.300,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
1,I-10869,1,BEEF STIRFRY COV FR,5.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
2,I-7064,1,BEEF OUTSIDE FLAT AAA,1.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
3,I-37005,1,BEEF MEATBALLS,4.540,Kg,1000.000,g,MEAT,1,beef & buffalo meat,41.34630
4,I-37002,1,BEEF INSIDE ROUND SHAVED,9.000,Kg,1000.000,g,MEAT,1,beef & buffalo meat,41.34630
5,I-3876,1,BEEF CHUCK GROUND AAA,1.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
6,I-34065,1,BEEF BONES KNUCKLE FZ,1.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
7,I-11661,1,"BEEF STEW 3/4"" FROZEN",5.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
8,I-3837,1,"BEEF STEW DICED 3/4""FR",1.000,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630
9,I-43648,1,BRISKET BEEF CORN SMKD FR,4.500,Kg,1.000,Kg,MEAT,1,beef & buffalo meat,41.34630


In [51]:
mapping.shape

(778, 11)

In [52]:
mapping.dtypes

ItemId                                                     object
CategoryID                                                  int64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [53]:
path = os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)